In [1]:
import numpy as np
import json

In [2]:
#vocabulary generate kore pura training corpus er

def generate_vocab(json_file_path, threshold=2):
    with open(json_file_path, "r" , encoding="utf-8") as json_file:
        json_data = json.load(json_file)

    vocab = {'<unk>': 0}
    counter = 0

    # each word occurs count kore
    for entry in json_data:
        sentence = entry["sentence"]
        counter+=1

        for word in sentence:
            for subword in word.split():
                if subword.strip():
                    if subword not in vocab:
                        vocab[subword] = 1
                    else:
                        vocab[subword] += 1

    # jegulo dataset e 1 bar hoese segulo count kore
    for k in list(vocab.keys())[1:]:
        if vocab[k] < threshold:
            vocab['<unk>'] += vocab[k]
            del vocab[k]
    #vocab['<unk>'] = 0
    # descending order e vocab sort kore
    sorted_vocab = {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)}

    # <unk> k top e push kore
    unk_val = sorted_vocab.pop('<unk>')
    sorted_vocab = {'<unk>': unk_val, **sorted_vocab}

    with open('vocab.txt', 'w', encoding="utf-8") as f:
        format_str = ''
        for index, (key, count) in enumerate(sorted_vocab.items()):
            format_str += key + '\t' + str(index) + '\t' + str(count) + '\n'
        f.write(format_str)

    #print("Threshold =", threshold)

    return sorted_vocab, counter

vocab, counter = generate_vocab("C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train_bangla.json")
print("Overall size of my vocabulary =", len(vocab))
#print("No. of times '<unk>' occurs in my vocabulary =", vocab['<unk>'])
print("No. of sentence =", counter)

Overall size of my vocabulary = 61
No. of sentence = 51


In [3]:
def load_training_data(file_path):
    with open(file_path, 'r', encoding="utf-8") as json_file:
        return json.load(json_file)

#transition holo NNP to VBP kotobar, emission holo "Book" word ta kotobar NNP r kotobar VBP
#unique state holo distinct tag gulo niye banano list
def process_training_data(data, vocab):
    transition_probabilities = {}
    emission_probabilities = {}
    unique_states = []

    for entry in data:
        sentence_tokens = entry['sentence']
        labels = entry['labels']
        prior_state = 'root'  

        for word, tag in zip(sentence_tokens, labels):
            if word not in vocab.keys():
                word = '<unk>'

            # Update unique states
            if tag not in unique_states:
                unique_states.append(tag)

            # Update transition count
            transition_key = (prior_state, tag)
            if transition_key not in transition_probabilities:
                transition_probabilities[transition_key] = 1
            else:
                transition_probabilities[transition_key] += 1

            # Update emission count
            emission_key = (tag, word)
            if emission_key not in emission_probabilities:
                emission_probabilities[emission_key] = 1
            else:
                emission_probabilities[emission_key] += 1

            prior_state = tag

    return transition_probabilities, emission_probabilities, unique_states

# normalize kore
def normalize_probabilities(probabilities):
    normalized_probabilities = {}
    for key in probabilities:
        state, value = key
        total = sum(v for k, v in probabilities.items() if k[0] == state)
        normalized_probabilities[key] = probabilities[key] / total
    return normalized_probabilities

#transition r emission gulo k dictionary er moto kore dekhano hoyese. hmm.json e ase sobgulo
def save_hmm_model(transition_probs, emission_probs, output_file):
    
    # Convert tuple keys to strings
    transition_probs = {str(k): v for k, v in transition_probs.items()}
    emission_probs = {str(k): v for k, v in emission_probs.items()}

    hmm_model = {
        'transition': transition_probs,
        'emission': emission_probs,
    }

    with open(output_file, 'w') as f:
        json.dump(hmm_model, f)

    return transition_probs, emission_probs

training_data = load_training_data('C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train_bangla.json')
transition_probs, emission_probs, unique_states = process_training_data(training_data, vocab)
normalized_transition_probs = normalize_probabilities(transition_probs)
normalized_emission_probs = normalize_probabilities(emission_probs)
transition_param_dict, emission_param_dict = save_hmm_model(normalized_transition_probs, normalized_emission_probs, 'hmm.json')

print("No. of transition parameters =", len(transition_param_dict))
print("No. of emission parameters =", len(emission_param_dict))


No. of transition parameters = 144
No. of emission parameters = 98


In [4]:
import nltk

In [5]:
# formatted_data er transition r emission dey 
def greedy_decoding(formatted_data, vocab, transition_param_dict, emission_param_dict, state_track):
    #accuracy = []
    transition_prob ={}
    emission_prob = {}
    #max_stt = {}

    prior_st = 'root'
    counter = 0
    for i in formatted_data:
        print(i)
        counter = counter+1
        transition_prob[counter]=transition_param_dict.get(str((prior_st, i[1])), 1e-7)
        emission_prob[counter]=emission_param_dict.get(str((i[1], i[0])), 0)
        prior_st = i[1]

    return transition_prob, emission_prob

In [6]:
def reverse_graph(G):
    '''reverse graph ta return kore like g[dst][src]=G[src][dst]'''
    g = {}
    for src in G.keys():
        for dst in G[src].keys():
            if dst not in g.keys():
                g[dst] = {}
            g[dst][src] = G[src][dst]
    return g


def build_max(rg, root): # rg always reverse_graph hbe
    '''max incoming edge find out kore each node er'''
    mg = {}
    for dst in rg.keys():
        if dst == root: # root thakle ignore krbe
            continue
        max_ind = -100
        max_value = -100
        for src in rg[dst].keys():
            if rg[dst][src] >= max_value:
                max_ind = src
                max_value = rg[dst][src]
        mg[dst] = {max_ind: max_value}
    return mg


def find_circle(mg):
    '''circle ase kina check kore DFS diye. na thakle None return kore'''

    for start in mg.keys():
        visited = [] # jegulo visited hoye giyese segulo ekhane thakbe
        stack = [start]
        while stack: #stack empty na hoa prjnto traverse cholbe
            n = stack.pop()
            if n in visited:# n jodi already visited hoy tahole cycle
                C = [] # empty list declare krlm
                while n not in C: # ekhane backtrack er kaj ta hoy loop diye
                    C.append(n) # n k C er moddhe rakhlm
                    n = list(mg[n].keys())[0] # n k update krbo tar parent node diye 
                return C # Cycle er node gulor list return kore
            visited.append(n)
            if n in mg.keys(): # n er kono child ase kina check kore
                stack.extend(list(mg[n].keys())) # stack er moddhe current node n er child k rakhe. etai DFS er condition
    return None # cycle na thakle None return kore


def chu_liu_edmond(G, root):
    
    # reversed graph rg[dst][src] = G[src][dst]
    rg = reverse_graph(G)
    # root er only out edge
    rg[root] = {}
    # the maximum edge select korlam for each node other than root
    mg = build_max(rg, root)

    # check if mg is a tree (contains a circle)
    C = find_circle(mg)
    # circle na thakle, mg tai max_spanning_tree
    if not C:
        return reverse_graph(mg)

    # jesob node circle kore tader k niye compact node korlm
    all_nodes = G.keys()
    vc = max(all_nodes) + 1

    # new graph holo G_prime
    #V_prime = list(set(all_nodes) - set(C)) + [vc]
    G_prime = {}
    vc_in_idx = {}
    vc_out_idx = {}
    # Now add the edges to G_prime
    for u in all_nodes:
        for v in G[u].keys():
            # incoming edge er weight calculation. subtraction hoy
            if (u not in C) and (v in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                w = G[u][v] - list(mg[v].values())[0] 
                if (vc not in G_prime[u]) or (vc in G_prime[u] and w > G_prime[u][vc]):
                    G_prime[u][vc] = w
                    vc_in_idx[u] = v

            # outgoing edge er weight calculation. jeta max oita nibo
            elif (u in C) and (v not in C):
                if vc not in G_prime.keys():
                    G_prime[vc] = {}
                w = G[u][v]
                if (v not in G_prime[vc]) or (v in G_prime[vc] and w > G_prime[vc][v]):
                    G_prime[vc][v] = w
                    vc_out_idx[v] = u

            # Third case: if the source and dest are all not in the circle, then just add the edge to the new graph.
            elif (u not in C) and (v not in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                G_prime[u][v] = G[u][v]

    # Recursively run the algorihtm on the new graph G_prime
    A = chu_liu_edmond(G_prime, root)
    # print(A)

    # compacted node k vangbo, erpor incoming r outgoing edge gulo identify krbo
    # always max ta choose krbo r bakigulo delete krbo
    all_nodes_A = list(A.keys())
    for src in all_nodes_A:
        # The number of out-edges varies, could be 0 or any number <=|V\C|
        if src == vc:
            for node_in in A[src].keys():
                orig_out = vc_out_idx[node_in]
                if orig_out not in A.keys():
                    A[orig_out] = {}
                A[orig_out][node_in] = G[orig_out][node_in]
        else:
            #for dst in A[src]:
            for dst in list(A[src].keys()):
                # There must be only one in-edge to vc.
                if dst == vc:
                    orig_in = vc_in_idx[src]
                    A[src][orig_in] = G[src][orig_in]
                    del A[src][dst]
    
    if vc in A:
        del A[vc]

    for node in C:
        if node != orig_in:
            src = list(mg[node].keys())[0]
            if src not in A.keys():
                A[src] = {}
            A[src][node] = mg[node][src]

    return A


In [7]:
# sentence separate kore
def separate_bangla_sentences(text):
    
    sentences = []
    current_sentence = ""
    end_sentence_marks = ['।', '?', '!']
    for char in text:
        current_sentence += char
        if char in end_sentence_marks:
            sentences.append(current_sentence.strip())
            current_sentence = ""
    if current_sentence:
        sentences.append(current_sentence.strip())
    
    return sentences

In [81]:
from bnlp import BengaliPOS

bn_pos = BengaliPOS()

def tree_generation(text):
    bangla_text = text
    dependency = []
    dependency_2 = []
    res_s = []
    dp_edges = []
    separated_sentences = separate_bangla_sentences(bangla_text)
    print(separated_sentences)

    for sentence in separated_sentences:
        print(sentence)
        res = bn_pos.tag(sentence)
        print(res)

        transition_prob, emission_prob = greedy_decoding(res, vocab, transition_param_dict, emission_param_dict, unique_states)
        print('Transition_prob = ', transition_prob)
        print('Emission_prob = ', emission_prob)

        res = ['root']+res
        res_s.append(res)
        print(res)

        def get_edges(graph):
            edges = []
            for node in graph:
                for neighbor in graph[node]:
                    edges.append((node, neighbor))
            return edges

        G = {}
        dp = {}
        print(len(res))
        for i in range(len(res)):
            G[i] = {}
            if i==0:
                p=500
                q=100
                r=50
                s=1
                for j in range(1, len(res)):
                    print(res[j])
                    weight = transition_prob[j] + emission_prob[j]
                    if res[j][1] == 'CCD' or res[j][1] == 'CSB':
                        weight = p*(transition_prob[j] + emission_prob[j])
                        p = p*3
                    if res[j][1] == 'VM' or res[j][1] == 'VAUX':
                        weight = q*(transition_prob[j] + emission_prob[j])
                        q = q/3
                    if res[j][1] == 'PPR':
                        weight = r*(transition_prob[j] + emission_prob[j])
                        r = r/3
                        
                    else:
                        weight = weight
                    G[i][j] = weight # done. r kono root add korbo na
                continue

            for j in range(len(res)):
                if (j == 0):
                    continue
                if (i == j):
                    continue
                weight = transition_prob[j] + emission_prob[j]
                if res[j][1] == 'NC':
                    weight = p*(transition_prob[j] + emission_prob[j])
                    p = p/3
                if res[j][1] == 'DAB':
                    weight = p*(transition_prob[j] + emission_prob[j])
                    p = p/3
                if res[j][1] == 'NP':
                    weight = p*(transition_prob[j] + emission_prob[j])
                    p = p/3

                G[i][j] = weight
                
        #print("G =", G)

        dp = chu_liu_edmond(G, 0)
        #dp = max_spanning_tree(G)
        #dp = chu_liu_edmond_without_reversal(G, 0)
        print('DP =',dp)
        edges_of_dp = get_edges(dp)
        dp_edges.append(get_edges(dp)) 

        # Define the list of tuples
        list_of_tuples = edges_of_dp
        word_tags = res
        edges = []
        for edge in list_of_tuples:
            src = word_tags[edge[0]][0] if edge[0] != 0 else 'root'
            dst = word_tags[edge[1]][0]
            edges.append((src, dst))
        print("List of Edges:", edges)
        dependency.append(edges)
        
        ########
        list_of_tuples_2 = edges_of_dp
        word_tags_2 = res
        edges_2 = []
        for edge in list_of_tuples_2:
            src = word_tags[edge[0]][1] if edge[0] != 0 else 'root'
            dst = word_tags[edge[1]][1]
            edges_2.append((src, dst))
        print("List of Edges_2:", edges_2)
        dependency_2.append(edges_2)
        

    return dependency, dependency_2, res_s, dp_edges, separated_sentences

input_sentence = "আমি ভাত খাই। আমি বাংলায় গান গাই।  সে বাজারে যায়। তুমি কি গান শুনবে? তিনি কি সত্যিই ভালো মানুষ? সে ধনী কিন্তু তার ভাই গরীব। প্রাকৃতিক রূপবৈচিত্র্যে ভরা আমাদের এই বাংলাদেশ। এই দেশে পরিচিত অপরিচিত অনেক পর্যটক-আকর্ষক স্থান আছে। এর মধ্যে প্রত্নতাত্ত্বিক নিদর্শন, ঐতিহাসিক মসজিদ এবং মিনার, পৃথিবীর দীর্ঘতম প্রাকৃতিক সমুদ্র সৈকত, পাহাড়, অরণ্য ইত্যাদি অন্যতম। এদেশের প্রাকৃতিক সৌন্দর্য পর্যটকদের মুগ্ধ করে। বাংলাদেশের প্রত্যেকটি এলাকা বিভিন্ন স্বতন্ত্র্র বৈশিষ্ট্যে বিশেষায়িত । পদ্মা বাংলাদেশের একটি প্রধান নদী। এটি হিমালয়ে উৎপন্ন গঙ্গানদীর প্রধান শাখা এবং বাংলাদেশের ২য় দীর্ঘতম নদী।বাংলাদেশের গুরুত্বপূর্ণ শহর রাজশাহী এই পদ্মার উত্তর তীরে অবস্থিত। রাজস্ব আদায়ের তুলনায় সরকারের ঋণের বোঝা বেড়ে চলেছে। পাখিগুলো নীল আকাশে উড়ছে। তিনি ভাত খেয়ে ঘুমিয়ে পড়লেন।তিনি খুব সকালে ঘুম থেকে উঠে পায়চারি করতে করতে বাজারের দিকে গেলেন । আমি পড়াশোনা শেষ করে খেলতে যাব।সে এখানে এল এবং সব কথা বলল।যে ছেলেটি এখানে এসেছিল, সে আমার ভাই।আমরা তিন ভাইবোন।ঢাকার প্রাচীন নাম জাহাঙ্গীরনগর।ঢাকা বাংলাদেশের রাজধানী।ধান ও গম বাংলাদেশের প্রধান দুটি খাদ্য শস্য।বাংলাদেশ রয়েল বেঙ্গল টাইগারের দেশ যার বাস সুন্দরবনে।সুন্দরবনে আছে বন্য প্রাণী এবং পৃথিবীখ্যাত ম্যানগ্রোভ ফরেস্ট এ বনাঞ্চলে অবস্থিত ।বাংলাদেশে প্রায় সব জায়গায় দোয়েল দেখতে পাওয়া যায়।জুমচাষেও ভুট্টার আবাদ হয়।কচুরিপানা একটি জলজ উদ্ভিদ।আমার পছন্দের খেলা ফুটবল।আমার পছন্দের রং লাল। আমি সকালে উঠে পানি পানি করি। সকালে আমি চা খাই। আমি সবসময় মা কে ভালোবাসি।পাখি গাছের উপরে বসে থাকে। আকাশে মেঘ পাওয়া যায়। ফুলের মধ্যে মধু থাকে। পানি হলো জীবনের উৎস। বায়ু আমাদের জীবনের জন্য প্রয়োজনীয়।পৃথিবী আমাদের একমাত্র বাসস্থান। পৃথিবীতে অনেক দেশ আছে। মোবাইলে খুব সহজেই ছবি তুলা যায়। মোবাইলে ইন্টারনেট চালানো হয়। বিজ্ঞানে অধ্যয়ন করা খুব মজাদার। রবীন্দ্রনাথ ঠাকুর একজন মহান কবি। গাছ আমাদের অক্সিজেন প্রদান করে।বাজারে এসে আমি আমার পছন্দের মিষ্টি কিনি। শিক্ষক আমাদের বই দেখায়। স্কুলে আমরা বই পড়ি। টাকা জরুরি ব্যবহারে রাখুন। নৌকা দেখে আমি খুব আনন্দিত হয়েছি।"
#input_sentence = "কচুরিপানা একটি জলজ উদ্ভিদ। আমার পছন্দের খেলা ফুটবল।"
dependencies, dependencies_2, res_sies, dp_edgesies, separated_sentencesies = tree_generation(input_sentence)

['আমি ভাত খাই।', 'আমি বাংলায় গান গাই।', 'সে বাজারে যায়।', 'তুমি কি গান শুনবে?', 'তিনি কি সত্যিই ভালো মানুষ?', 'সে ধনী কিন্তু তার ভাই গরীব।', 'প্রাকৃতিক রূপবৈচিত্র্যে ভরা আমাদের এই বাংলাদেশ।', 'এই দেশে পরিচিত অপরিচিত অনেক পর্যটক-আকর্ষক স্থান আছে।', 'এর মধ্যে প্রত্নতাত্ত্বিক নিদর্শন, ঐতিহাসিক মসজিদ এবং মিনার, পৃথিবীর দীর্ঘতম প্রাকৃতিক সমুদ্র সৈকত, পাহাড়, অরণ্য ইত্যাদি অন্যতম।', 'এদেশের প্রাকৃতিক সৌন্দর্য পর্যটকদের মুগ্ধ করে।', 'বাংলাদেশের প্রত্যেকটি এলাকা বিভিন্ন স্বতন্ত্র্র বৈশিষ্ট্যে বিশেষায়িত ।', 'পদ্মা বাংলাদেশের একটি প্রধান নদী।', 'এটি হিমালয়ে উৎপন্ন গঙ্গানদীর প্রধান শাখা এবং বাংলাদেশের ২য় দীর্ঘতম নদী।', 'বাংলাদেশের গুরুত্বপূর্ণ শহর রাজশাহী এই পদ্মার উত্তর তীরে অবস্থিত।', 'রাজস্ব আদায়ের তুলনায় সরকারের ঋণের বোঝা বেড়ে চলেছে।', 'পাখিগুলো নীল আকাশে উড়ছে।', 'তিনি ভাত খেয়ে ঘুমিয়ে পড়লেন।', 'তিনি খুব সকালে ঘুম থেকে উঠে পায়চারি করতে করতে বাজারের দিকে গেলেন ।', 'আমি পড়াশোনা শেষ করে খেলতে যাব।', 'সে এখানে এল এবং সব কথা বলল।', 'যে ছেলেটি এখানে এসেছিল, সে আমার ভাই।', 'আমরা তিন ভাইবোন।', 

In [93]:
print(dependencies[30])

[('root', 'আমার'), ('আমার', 'পছন্দের'), ('আমার', 'খেলা'), ('আমার', 'ফুটবল'), ('ফুটবল', '।')]


In [95]:
print(dependencies[29])

[('root', 'একটি'), ('কচুরিপানা', 'জলজ'), ('কচুরিপানা', 'উদ্ভিদ'), ('একটি', 'কচুরিপানা'), ('উদ্ভিদ', '।')]


In [70]:
print(dependencies_2)

[[('root', 'VM'), ('root', 'NC'), ('VM', 'PU'), ('NC', 'NC'), ('NC', 'NC')]]


In [71]:
print(dp_edgesies)

[[(0, 4), (0, 1), (4, 5), (1, 3), (1, 2)]]


In [92]:
print(separated_sentencesies[30])

আমার পছন্দের খেলা ফুটবল।


In [96]:
print(separated_sentencesies[29])

কচুরিপানা একটি জলজ উদ্ভিদ।


In [73]:
# eita only ekta sentence er jnno
'''
def load_training_data_2(file_path):
    with open(file_path, 'r', encoding="utf-8") as json_file:
        return json.load(json_file)
def process_training_data_2(data):
    correct_count = 0
    for entry in data:
        sentence_tokens = entry['sentence']
        labels = entry['edges']
        if input_sentence == sentence_tokens:
            #print(sentence_tokens)
            #print(input_sentence)
            #print(labels)
            total_count = len(dependencies[0])
            #print(total_count)
            for dependency in dependencies[0]:
                dependency_list = list(dependency)  # Convert tuple to list
                #print(dependency_list)
                if dependency_list in labels:
                    correct_count += 1

            accuracy = correct_count / total_count if total_count > 0 else 0
            #print(accuracy)
    return accuracy

training_data_2 = load_training_data_2('C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/dev_bangla_accuracy.json')
accuracy_2 = process_training_data_2(training_data_2)
print(accuracy_2)
'''

'\ndef load_training_data_2(file_path):\n    with open(file_path, \'r\', encoding="utf-8") as json_file:\n        return json.load(json_file)\ndef process_training_data_2(data):\n    correct_count = 0\n    for entry in data:\n        sentence_tokens = entry[\'sentence\']\n        labels = entry[\'edges\']\n        if input_sentence == sentence_tokens:\n            #print(sentence_tokens)\n            #print(input_sentence)\n            #print(labels)\n            total_count = len(dependencies[0])\n            #print(total_count)\n            for dependency in dependencies[0]:\n                dependency_list = list(dependency)  # Convert tuple to list\n                #print(dependency_list)\n                if dependency_list in labels:\n                    correct_count += 1\n\n            accuracy = correct_count / total_count if total_count > 0 else 0\n            #print(accuracy)\n    return accuracy\n\ntraining_data_2 = load_training_data_2(\'C:/Users/Admin/Desktop/Thesis/POS Ta

In [94]:
# eita pura paragraph er jnno
def load_training_data_3(file_path):
    with open(file_path, 'r', encoding="utf-8") as json_file:
        return json.load(json_file)
def process_training_data_3(data):
    #correct_count = 0
    #print(separated_sentencesies[0])
    accuracy_3= []
    for i in range (len(separated_sentencesies)):
        correct_count = 0
        for entry in data:
            sentence_tokens = entry['sentence']
            labels = entry['edges']
            if separated_sentencesies[i] == sentence_tokens:
                #print(sentence_tokens)
                print(separated_sentencesies[i])
                total_count = len(dependencies[i])
                #print(total_count)
                for dependency in dependencies[i]:
                    dependency_list = list(dependency)
                    #print(dependency_list)
                    if dependency_list in labels:
                        correct_count += 1

                accuracy = correct_count / total_count if total_count > 0 else 0
                #print(accuracy)
                accuracy_3.append(accuracy)
    return accuracy_3

training_data_3 = load_training_data_3('C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/dev_bangla_accuracy.json')
accuracy_3 = process_training_data_3(training_data_3)
print(accuracy_3)
mean =0
for i in accuracy_3:
    mean = mean + i
print("mean accuracy :", mean/len(accuracy_3))
mini = min(accuracy_3)
maxi = max(accuracy_3)
print("minimum :", mini)
print(accuracy_3.index(mini))
print("maximum :", maxi)
print(accuracy_3.index(maxi))
#print(separated_sentencesies[29])
#print(separated_sentencesies[30])

আমি ভাত খাই।
আমি বাংলায় গান গাই।
সে বাজারে যায়।
তুমি কি গান শুনবে?
তিনি কি সত্যিই ভালো মানুষ?
সে ধনী কিন্তু তার ভাই গরীব।
এই দেশে পরিচিত অপরিচিত অনেক পর্যটক-আকর্ষক স্থান আছে।
এর মধ্যে প্রত্নতাত্ত্বিক নিদর্শন, ঐতিহাসিক মসজিদ এবং মিনার, পৃথিবীর দীর্ঘতম প্রাকৃতিক সমুদ্র সৈকত, পাহাড়, অরণ্য ইত্যাদি অন্যতম।
এদেশের প্রাকৃতিক সৌন্দর্য পর্যটকদের মুগ্ধ করে।
বাংলাদেশের প্রত্যেকটি এলাকা বিভিন্ন স্বতন্ত্র্র বৈশিষ্ট্যে বিশেষায়িত ।
পদ্মা বাংলাদেশের একটি প্রধান নদী।
এটি হিমালয়ে উৎপন্ন গঙ্গানদীর প্রধান শাখা এবং বাংলাদেশের ২য় দীর্ঘতম নদী।
বাংলাদেশের গুরুত্বপূর্ণ শহর রাজশাহী এই পদ্মার উত্তর তীরে অবস্থিত।
রাজস্ব আদায়ের তুলনায় সরকারের ঋণের বোঝা বেড়ে চলেছে।
পাখিগুলো নীল আকাশে উড়ছে।
তিনি ভাত খেয়ে ঘুমিয়ে পড়লেন।
তিনি খুব সকালে ঘুম থেকে উঠে পায়চারি করতে করতে বাজারের দিকে গেলেন ।
আমি পড়াশোনা শেষ করে খেলতে যাব।
সে এখানে এল এবং সব কথা বলল।
যে ছেলেটি এখানে এসেছিল, সে আমার ভাই।
ঢাকার প্রাচীন নাম জাহাঙ্গীরনগর।
ঢাকা বাংলাদেশের রাজধানী।
ধান ও গম বাংলাদেশের প্রধান দুটি খাদ্য শস্য।
বাংলাদেশ রয়েল বেঙ্গল টাইগারের দ

In [65]:
my_list = [10, 5, 8, 4, 12]

# Find the minimum value and its position
min_value = min(my_list)
min_position = my_list.index(min_value)

# Print the minimum value and its position
print("Minimum value:", min_value)
print("Position of minimum value:", min_position)
print(my_list[min_position])

Minimum value: 4
Position of minimum value: 3
4


In [15]:
#print(transition_param_dict)

In [16]:
def get_keys_and_values_with_string_at_position_0(dictionary, string):
    key_value_tuples = []
    for key, value in dictionary.items():
        if key.startswith(f"('{string}'"):
            key_value_tuples.append(((string, key.split(", ")[1][1:-2]), value))
    return key_value_tuples


In [17]:
def accuracy_measure(dependencies_2):
    tuple_sort_accuracy_list = []
    for sent in dependencies_2:
        tuple_sort_accuracy = 0
        print(len(sent))
        for tuples in sent:
            #print(tuples)
            result = get_keys_and_values_with_string_at_position_0(transition_param_dict, tuples[0])
            sorted_result = sorted(result, key=lambda x: x[1], reverse=True)
            #print(sorted_result)
            #print(len(sorted_result))
            x = 0
            p = 0
            for tuple_sort in sorted_result:
                #print(tuple_sort)
                x = x+1 
                if(tuples == tuple_sort[0]):
                    print("tuple_sort:", tuple_sort)
                    print("x :", x)
                    tuple_sort_accuracy = tuple_sort_accuracy + (tuple_sort[1])/len(sent)*x
                    print("tuple_sort accuracy:", tuple_sort_accuracy)
                    p = 1
                
            if (p == 0):
                print("sorted result:", len(sorted_result))
                #q = 0.02
                if tuple_sort_accuracy > .7 or len(sorted_result) >= 10:
                    tuple_sort_accuracy = tuple_sort_accuracy + len(sorted_result)* .01
                else:
                    tuple_sort_accuracy = tuple_sort_accuracy + len(sorted_result)* .02
                print("tuple_sort accuracyy:", tuple_sort_accuracy)
        tuple_sort_accuracy_list.append(tuple_sort_accuracy)
        print("tuple_sort accuracy for a sentence:", tuple_sort_accuracy)
    #print("tuple_sort accuracy for a sentence:", tuple_sort_accuracy_list)
    return tuple_sort_accuracy_list
print("tuple_sort accuracy for a sentence:", accuracy_measure(dependencies_2))


9
tuple_sort: (('root', 'VM'), 0.0392156862745098)
x : 9
tuple_sort accuracy: 0.0392156862745098
tuple_sort: (('root', 'VM'), 0.0392156862745098)
x : 9
tuple_sort accuracy: 0.0784313725490196
tuple_sort: (('root', 'NC'), 0.2549019607843137)
x : 1
tuple_sort accuracy: 0.10675381263616557
tuple_sort: (('PU', 'JQ'), 0.022727272727272728)
x : 7
tuple_sort accuracy: 0.12443058031293325
sorted result: 10
tuple_sort accuracyy: 0.22443058031293325
tuple_sort: (('VM', 'PU'), 0.5357142857142857)
x : 1
tuple_sort accuracy: 0.28395438983674276
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
x : 1
tuple_sort accuracy: 0.3192820251643781
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
x : 1
tuple_sort accuracy: 0.3546096604920134
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
x : 1
tuple_sort accuracy: 0.38993729581964875
tuple_sort accuracy for a sentence: 0.38993729581964875
5
tuple_sort: (('root', 'JJ'), 0.0784313725490196)
x : 4
tuple_sort accuracy: 0.06274509803921569
tuple_sort: (('NC', 'NC')

In [18]:
'''
def accuracy_measure_2(dependencies_2):
    tuple_sort_accuracy_list = []
    for sent in dependencies_2:
        tuple_sort_accuracy = 0
        len_sen = len(sent)
        print(len(sent))
        tuple_sort_accuracy = 0
        for tuples in sent:
            #print(tuples)
            result = get_keys_and_values_with_string_at_position_0(transition_param_dict, tuples[0])
            sorted_result = sorted(result, key=lambda x: x[1], reverse=True)
            #print(sorted_result)
            #print(len(sorted_result))
            x = 0
            for tuple_sort in sorted_result:
                #print(tuple_sort[0])
                x = x+1
                if(tuples == tuple_sort[0]):
                    print("tuple_sort:", tuple_sort)
                    tuple_sort_accuracy = tuple_sort_accuracy + 1
                    print("tuple_sort accuracy:", tuple_sort_accuracy)
                    
        tuple_sort_accuracy = tuple_sort_accuracy/len(sent)
        tuple_sort_accuracy_list.append(tuple_sort_accuracy)
        print("tuple_sort accuracy for a sentence:", tuple_sort_accuracy)
    #print("tuple_sort accuracy for a sentence:", tuple_sort_accuracy_list)
    return tuple_sort_accuracy_list
print("tuple_sort accuracy for a sentence:", accuracy_measure_2(dependencies_2))
'''

9
tuple_sort: (('root', 'VM'), 0.0392156862745098)
tuple_sort accuracy: 1
tuple_sort: (('root', 'VM'), 0.0392156862745098)
tuple_sort accuracy: 2
tuple_sort: (('root', 'NC'), 0.2549019607843137)
tuple_sort accuracy: 3
tuple_sort: (('PU', 'JQ'), 0.022727272727272728)
tuple_sort accuracy: 4
tuple_sort: (('VM', 'PU'), 0.5357142857142857)
tuple_sort accuracy: 5
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
tuple_sort accuracy: 6
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
tuple_sort accuracy: 7
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
tuple_sort accuracy: 8
tuple_sort accuracy for a sentence: 0.8888888888888888
5
tuple_sort: (('root', 'JJ'), 0.0784313725490196)
tuple_sort accuracy: 1
tuple_sort: (('NC', 'NC'), 0.31794871794871793)
tuple_sort accuracy: 2
tuple_sort: (('NC', 'NP'), 0.010256410256410256)
tuple_sort accuracy: 3
tuple_sort: (('JJ', 'NC'), 0.7209302325581395)
tuple_sort accuracy: 4
tuple_sort: (('NP', 'PU'), 0.6)
tuple_sort accuracy: 5
tuple_sort accuracy for a sent

In [19]:
#print(emission_param_dict)

In [20]:
print(unique_states)

['RDF', 'NC', 'JJ', 'CCD', 'NV', 'PU', 'NP', 'VM', 'JQ', 'PP', 'PPR', 'VAUX', 'NST', 'PRL', 'AMN', 'CCL', 'CSB', 'DAB', 'CX', 'EX', 'MD', 'VB', 'NNS', 'VBG', 'WP', 'PRP', 'VBD', '.', "''", 'ALC', 'PWH', 'PRF', 'NNP']


In [36]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Example NLP function
def analyze_text(text):
    # Replace this with your actual NLP processing logic
    # For demonstration, let's just return the input text
    dependencies, dependencies_2, res_sies, dp_edgesies, separated_sentencesies = tree_generation(text)
    #accuracy = accuracy_measure(dependencies_2)
    accuracy = process_training_data_3(training_data_3)
    return {"title": "For Bangla", "sentence": text, "pos": res_sies, "input": dependencies, "num": dp_edgesies, "accu": accuracy}

@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        user_input = request.form['user_input']
        analysis = analyze_text(user_input)
        return render_template('new_result.html', analysis=analysis)
    return render_template('new.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Feb/2024 20:59:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2024 21:05:49] "POST / HTTP/1.1" 200 -


['পৃথিবী আমাদের একমাত্র বাসস্থান।', 'পৃথিবীতে অনেক দেশ আছে।', 'মোবাইলে খুব সহজেই ছবি তুলা যায়।', 'মোবাইলে ইন্টারনেট চালানো হয়।', 'বিজ্ঞানে অধ্যয়ন করা খুব মজাদার।', 'রবীন্দ্রনাথ ঠাকুর একজন মহান কবি।', 'গাছ আমাদের অক্সিজেন প্রদান করে।']
পৃথিবী আমাদের একমাত্র বাসস্থান।
[('পৃথিবী', 'NP'), ('আমাদের', 'PPR'), ('একমাত্র', 'JQ'), ('বাসস্থান', 'NC'), ('।', 'PU')]
('পৃথিবী', 'NP')
('আমাদের', 'PPR')
('একমাত্র', 'JQ')
('বাসস্থান', 'NC')
('।', 'PU')
Transition_prob =  {1: 0.0784313725490196, 2: 1e-07, 3: 0.1, 4: 0.56, 5: 0.15384615384615385}
Emission_prob =  {1: 0, 2: 0.1, 3: 0, 4: 0, 5: 0.4838709677419355}
['root', ('পৃথিবী', 'NP'), ('আমাদের', 'PPR'), ('একমাত্র', 'JQ'), ('বাসস্থান', 'NC'), ('।', 'PU')]
6
('পৃথিবী', 'NP')
('আমাদের', 'PPR')
('একমাত্র', 'JQ')
('বাসস্থান', 'NC')
('।', 'PU')
DP = {2: {1: 13.071895424836601}, 0: {2: 5.000005000000001}, 5: {3: 0.1}, 1: {4: 280.0}, 4: {5: 0.6377171215880894}}
List of Edges: [('আমাদের', 'পৃথিবী'), ('root', 'আমাদের'), ('।', 'একমাত্র'), ('পৃথিবী', 'বাসস্থ

127.0.0.1 - - [28/Feb/2024 21:12:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2024 21:16:48] "POST / HTTP/1.1" 200 -


['বাজারে এসে আমি আমার পছন্দের মিষ্টি কিনি।', 'শিক্ষক আমাদের বই দেখায়।', 'স্কুলে আমরা বই পড়ি।', 'টাকা জরুরি ব্যবহারে রাখুন।', 'নৌকা দেখে আমি খুব আনন্দিত হয়েছি।']
বাজারে এসে আমি আমার পছন্দের মিষ্টি কিনি।
[('বাজারে', 'NC'), ('এসে', 'VM'), ('আমি', 'PPR'), ('আমার', 'PPR'), ('পছন্দের', 'NC'), ('মিষ্টি', 'NC'), ('কিনি', 'VM'), ('।', 'PU')]
('বাজারে', 'NC')
('এসে', 'VM')
('আমি', 'PPR')
('আমার', 'PPR')
('পছন্দের', 'NC')
('মিষ্টি', 'NC')
('কিনি', 'VM')
('।', 'PU')
Transition_prob =  {1: 0.2549019607843137, 2: 0.16923076923076924, 3: 0.07142857142857142, 4: 1e-07, 5: 0.45, 6: 0.31794871794871793, 7: 0.16923076923076924, 8: 0.5357142857142857}
Emission_prob =  {1: 0, 2: 0, 3: 0.1, 4: 0.075, 5: 0, 6: 0, 7: 0, 8: 0.4838709677419355}
['root', ('বাজারে', 'NC'), ('এসে', 'VM'), ('আমি', 'PPR'), ('আমার', 'PPR'), ('পছন্দের', 'NC'), ('মিষ্টি', 'NC'), ('কিনি', 'VM'), ('।', 'PU')]
9
('বাজারে', 'NC')
('এসে', 'VM')
('আমি', 'PPR')
('আমার', 'PPR')
('পছন্দের', 'NC')
('মিষ্টি', 'NC')
('কিনি', 'VM')
('।', 'PU')
D

In [22]:
def format_data(json_file_path):
    with open(json_file_path, 'r', encoding="utf-8") as json_file:
        data = json.load(json_file)

    formatted_data = []

    for entry in data:
        sentence_tokens = entry['sentence']
        labels = entry['labels']

        tokens_with_index = []

        for index, (word, tag) in enumerate(zip(sentence_tokens, labels)):
            tokens_with_index.append(f"{index}\t{word}\t{tag}")

        sentence = '\n'.join(tokens_with_index)
        formatted_data.append(sentence)

    return '\n\n'.join(formatted_data)

In [23]:
def greedy_decoding_accuracy(formatted_data, vocab, transition_param_dict, emission_param_dict, state_track):
    accuracy = []

    for line in formatted_data.split('\n\n'):
        prior_st = 'root'
        for ent in line.split('\n'):
            index, word, st = ent.split('\t')
            if word not in vocab.keys():
                word = '<unk>'

            max_prob = 0
            max_st = None

            for st_ in state_track:
                tr = transition_param_dict.get(str((prior_st, st_)), 1e-7)
                em = emission_param_dict.get(str((st_, word)), 0)
                prob = tr * em

                if prob > max_prob:
                    max_prob = prob
                    max_st = st_

            accuracy.append(st == max_st)
            prior_st = max_st
            #print(prior_st)

    return sum(accuracy) / len(accuracy)

formatted_dev_data = format_data('C:/Users/Admin/Desktop/Thesis/POS Tagging/dev_bangla.json')
accuracy = greedy_decoding_accuracy(formatted_dev_data, vocab, transition_param_dict, emission_param_dict, unique_states)
print("Accuracy on dev data with greedy decoding =", accuracy)

Accuracy on dev data with greedy decoding = 0.7692307692307693


In [24]:
print(unique_states)

['RDF', 'NC', 'JJ', 'CCD', 'NV', 'PU', 'NP', 'VM', 'JQ', 'PP', 'PPR', 'VAUX', 'NST', 'PRL', 'AMN', 'CCL', 'CSB', 'DAB', 'CX', 'EX', 'MD', 'VB', 'NNS', 'VBG', 'WP', 'PRP', 'VBD', '.', "''", 'ALC', 'PWH', 'PRF', 'NNP']
